In [1]:
import ROOT as rt
import csv
import re
import sys
import collections
from collections import OrderedDict
import uproot
import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
from numpy import linalg as LA
from ROOT import TLorentzVector

import math

# sys.path.append('/home/cms/Desktop/LLP/cms_lpc_llp/jet_timing_studies/python/lib')
sys.path.append('/nfshome/christiw/llp/cms_lpc_llp/jet_timing_studies/python/lib')
# sys.path.append('/nfshome/christiw/llp/cms_lpc_llp/llp_analyzer/python/lib')

from histo_utilities import create_TH1D, create_TH2D, std_color_list
     
import CMS_lumi, tdrstyle 
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 1
CMS_lumi.extraText = "     Simulation Preliminary"

# donotdelete = []
print(sys.version)

Welcome to JupyROOT 6.16/00
3.6.6 (default, Mar 29 2019, 00:03:27) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-36)]


In [48]:
fpath_bkg =OrderedDict()
tree_bkg = OrderedDict()
tree_sig = OrderedDict()
fpath_sig =OrderedDict()

bkg_path = "/home/cms/Desktop/LLP/delayed_jet_analyzer/"
bkg_path = "/nfshome/christiw/llp/jet_timing_studies_root/"
bkg_path = "/nfshome/christiw/llp/jet_timing_studies_root/hcal_matching_fixed/"
lumi = 137000 #in pb-1
fpath_bkg['QCD'] = bkg_path+"jet_timing_studies_ntuple_RunIIFall17DRPremix_QCD_Pt_170to300_TuneCP5_13TeV_pythia8.root"
fpath_bkg['WJetsToLNu'] = bkg_path+"jet_timing_studies_ntuple_RunIIFall17DRPremix_WJetsToLNu_TuneCP5_13TeV-madgraphMLM-pythia8.root"
fpath_bkg['bbbb_ggh_mh125_ctau1'] = bkg_path+'jet_timing_studies_ntuple_bbbb_ggh_ISR_mh125_mx50_pl1000_ev100000.root'
fpath_bkg['bbbb_mh125_ctau1'] = bkg_path+'jet_timing_studies_ntuple_bbbb_vh_ISR_mh125_mx50_pl1000_ev100000.root'
fpath_bkg['bbbb_mh125_ctau10'] = bkg_path+'jet_timing_studies_ntuple_bbbb_vh_ISR_mh125_mx50_pl10000_ev100000.root'
fpath_bkg['bbbb_mh2000_ctau1'] = bkg_path+'jet_timing_studies_ntuple_bbbb_vh_ISR_mh2000_mx975_pl1000_ev100000.root'
fpath_bkg['bbbb_mh2000_ctau10'] = bkg_path+'jet_timing_studies_ntuple_bbbb_vh_ISR_mh2000_mx975_pl10000_ev100000.root'
fpath_bkg['gluino_m1000_ctau1'] = bkg_path+'jet_timing_studies_ntuple_gluinoGMSB_M1000_ctau1000p0.root'
fpath_bkg['gluino_m1000_ctau10'] = bkg_path+'jet_timing_studies_ntuple_gluinoGMSB_M1000_ctau10000p0.root'
fpath_bkg['bbbb_mh125_mx55_ctau1'] = bkg_path+'jet_timing_studies_ntuple_WminusH_HToSSTobbbb_mh125_mx55_pl1000.root'


llp_mass = {}
llp_mass['bbbb_mh125_ctau1'] = 50
llp_mass['bbbb_mh125_ctau10'] = 50
llp_mass['bbbb_ggh_mh125_ctau1'] = 50
llp_mass['bbbb_mh2000_ctau1'] = 975
llp_mass['bbbb_mh2000_ctau10'] = 975
llp_mass['bbbb_mh125_mx55_ctau1'] = 55
llp_mass['gluino_m1000_ctau1'] = 1000
llp_mass['gluino_m1000_ctau10'] = 1000


for k,v in fpath_bkg.items():
    print (k, v)
#     tree[k] = rtnp.root2array(v)
    root_dir = uproot.open(v) 
    tree_bkg[k] = root_dir['ntuples']['llp']
#     tree[k] = root_dir['ntuples']
    a = tree_bkg[k]["jetPt"].array()
    print (a[:-1][0])
    print(a.shape)

QCD /nfshome/christiw/llp/jet_timing_studies_root/hcal_matching_fixed/jet_timing_studies_ntuple_RunIIFall17DRPremix_QCD_Pt_170to300_TuneCP5_13TeV_pythia8.root
[171.15874  145.38046   43.887985]
(191509,)
WJetsToLNu /nfshome/christiw/llp/jet_timing_studies_root/hcal_matching_fixed/jet_timing_studies_ntuple_RunIIFall17DRPremix_WJetsToLNu_TuneCP5_13TeV-madgraphMLM-pythia8.root
[59.339394 27.090656 22.657576]
(189261,)
bbbb_ggh_mh125_ctau1 /nfshome/christiw/llp/jet_timing_studies_root/hcal_matching_fixed/jet_timing_studies_ntuple_bbbb_ggh_ISR_mh125_mx50_pl1000_ev100000.root
[38.70309  30.284447]
(50918,)
bbbb_mh125_ctau1 /nfshome/christiw/llp/jet_timing_studies_root/hcal_matching_fixed/jet_timing_studies_ntuple_bbbb_vh_ISR_mh125_mx50_pl1000_ev100000.root
[64.42451  51.139175 37.67556 ]
(74588,)
bbbb_mh125_ctau10 /nfshome/christiw/llp/jet_timing_studies_root/hcal_matching_fixed/jet_timing_studies_ntuple_bbbb_vh_ISR_mh125_mx50_pl10000_ev100000.root
[30.621801]
(82611,)
bbbb_mh2000_ctau1 /nfs

In [23]:
trigger_names_file = '/home/cms/Desktop/LLP/delayed_jet_analyzer/data/trigger_names_llp_v1.dat'
trigger_names_file = '/nfshome/christiw/llp/cms_lpc_llp/jet_timing_studies/data/trigger_names_llp_v1.dat'

trigger_names = []
with open(trigger_names_file) as f:
    reader = csv.reader(f, delimiter=" ")
    for line in reader:
        trigger_names.append(line[2])
    trigger_paths = [87,135] #PFMET120


# 177 HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ
# 362 HLT_Ele23_Ele12_CaloIdL_TrackIdL_IsoVL_DZ
# 310 HLT_PFMET120_PFMHT120_IDTight
# 87 HLT_Ele32_WPTight_Gsf
# 135 HLT_IsoMu24
# 310 PFMET

In [24]:
def deltaPhi( phi1,  phi2):

    dphi = phi1-phi2
    while (dphi > math.pi):
        dphi -= 2* math.pi
    while (dphi <= -math.pi):
        dphi += 2* math.pi
    return dphi


In [50]:
# Gen level studies
gen_time_matched = {}


gen_time = {}
decay_radius = {}
delay_beta = {}
delay_path = {}
gLLP_daughter_deltaR = {}
llp_eta = {}
llp_phi = {}
llp_beta = {}
angle = {} # dot product between LLP and LLP daughter
rf_angle = {}
llp_p_lf = {} # lab frame
llp_p_hf = {} #higgs frame
higgs_pt = {}
for k,T in tree_bkg.items():
    # selections
#     if k == 'bbbb_ggh_mh125_ctau1':
    if k[:4] == 'glui' or k[:4]=='bbbb':
        eventNum = T['eventNum'].array()
        gen_time_var = T['gen_time'].array()

        gParticleId = T['gParticleId'].array()
        gParticlePt = T['gParticlePt'].array()
        gParticlePhi = T['gParticlePhi'].array()
        gParticleEta = T['gParticleEta'].array()
        gParticleE = T['gParticleE'].array()
        gParticleStatus = T['gParticleStatus'].array()
        
        gParticleMotherId = T['gParticleMotherId'].array()
        gParticleMotherIndex = T['gParticleMotherIndex'].array()

        gLLP_daughter_match_jet_index = T['gLLP_daughter_match_jet_index'].array()


        gLLP_daughter_travel_time = T['gLLP_daughter_travel_time'].array()
        photon_travel_time = T['photon_travel_time'].array()
        gLLP_decay_radius = np.sqrt(T['gLLP_decay_vertex_x'].array()**2+T['gLLP_decay_vertex_y'].array()**2)
        gLLP_travel_time = T['gLLP_travel_time'].array()

        gLLP_beta = T['gLLP_beta'].array()

        genVertexT = T['genVertexT'].array()
        gLLP_daughter_eta = T['gLLP_daughter_eta'].array()
        gLLP_daughter_phi = T['gLLP_daughter_phi'].array()
        gLLP_daughter_pt = T['gLLP_daughter_pt'].array()
        gLLP_daughter_e = T['gLLP_daughter_e'].array()


        genVertexY = np.column_stack((T['genVertexY'].array(),T['genVertexY'].array()))
        genVertexX = np.column_stack((T['genVertexX'].array(),T['genVertexX'].array()))
        genVertexZ = np.column_stack((T['genVertexZ'].array(),T['genVertexZ'].array()))
        gLLP_decay_vertex_x = T['gLLP_decay_vertex_x'].array()
        gLLP_decay_vertex_y = T['gLLP_decay_vertex_y'].array()
        gLLP_decay_vertex_z = T['gLLP_decay_vertex_z'].array()

        if k[:4] == 'glui':
            gLLP_travel_time = ((gLLP_decay_vertex_x-genVertexX)**2+(gLLP_decay_vertex_y-genVertexY)**2+(gLLP_decay_vertex_z-genVertexZ)**2)**0.5/(30.*gLLP_beta)
            gen_time_var = np.column_stack((gLLP_travel_time[:,0],gLLP_travel_time[:,0],gLLP_travel_time[:,1],gLLP_travel_time[:,1])) + gLLP_daughter_travel_time - photon_travel_time
        gLLP_e = T['gLLP_e'].array()
        gLLP_pt = T['gLLP_pt'].array()
        gLLP_phi = T['gLLP_phi'].array()
        gLLP_eta = T['gLLP_eta'].array()
        LLP_daughter_angle = np.cos(np.column_stack((gLLP_phi[:,0],gLLP_phi[:,0],gLLP_phi[:,1],gLLP_phi[:,1])))*np.cos(gLLP_daughter_phi)+np.sin(np.column_stack((gLLP_phi[:,0],gLLP_phi[:,0],gLLP_phi[:,1],gLLP_phi[:,1])))*np.sin(gLLP_daughter_phi)+np.sinh(np.column_stack((gLLP_eta[:,0],gLLP_eta[:,0],gLLP_eta[:,1],gLLP_eta[:,1])))*np.sinh(gLLP_daughter_eta)
        LLP_daughter_angle = LLP_daughter_angle/np.sqrt(np.cosh(np.column_stack((gLLP_eta[:,0],gLLP_eta[:,0],gLLP_eta[:,1],gLLP_eta[:,1])))**2*np.cosh(gLLP_daughter_eta)**2)
        #dot product between gLLP and gLLP daughter
        gLLP_beta_x = gLLP_beta*np.cos(gLLP_phi)/np.cosh(gLLP_eta)
        gLLP_beta_y = gLLP_beta*np.sin(gLLP_phi)/np.cosh(gLLP_eta)
        gLLP_beta_z = gLLP_beta*np.sinh(gLLP_eta)/np.cosh(gLLP_eta)
        
        gen_time[k] = []
        gen_time_matched[k] = []
        decay_radius[k] = []
        delay_beta[k] = []
        delay_path[k] = []
        gLLP_daughter_deltaR[k] = []
        llp_eta[k] = []
        llp_phi[k] = []
        llp_beta[k] = []
        angle[k] = []
        rf_angle[k] = []
        higgs_pt[k] = []
        llp_p_hf[k] = []
        llp_p_lf[k] = []
        for ev in range(len(gLLP_beta_z)):
            if k[:7] == 'bbbb_mh': #vh
                mu = np.logical_and(np.logical_and(np.absolute(gParticleId[ev]) == 13, np.absolute(gParticleMotherId[ev]) == 24), gParticleMotherIndex[ev] == 1 )
                ele = np.logical_and(np.logical_and(np.absolute(gParticleId[ev]) == 11, np.absolute(gParticleMotherId[ev]) == 24), gParticleMotherIndex[ev] == 1 )
                if not (np.count_nonzero(mu) ==1 or np.count_nonzero(ele)==1): continue
            if k[:4] == 'bbbb':
                if 'ggh' in k:
                    llp1_sel = np.logical_and(gParticleId[ev] == 9000006,gParticleStatus[ev] == 22)
                    llp2_sel = np.logical_and(gParticleId[ev] == 9000007,gParticleStatus[ev] == 22)
                    assert(np.count_nonzero(llp1_sel)==np.count_nonzero(llp2_sel)==1)
                    llp1 = TLorentzVector()
                    llp1.SetPtEtaPhiE(gParticlePt[ev][llp1_sel],gParticleEta[ev][llp1_sel],gParticlePhi[ev][llp1_sel],gParticleE[ev][llp1_sel])
                    llp2 = TLorentzVector()
                    llp2.SetPtEtaPhiE(gParticlePt[ev][llp2_sel],gParticleEta[ev][llp2_sel],gParticlePhi[ev][llp2_sel],gParticleE[ev][llp2_sel])

                    higgs = llp1 + llp2
                    higgs_pt[k].append(higgs.Pt())

                else:
                    higgs_sel = abs(gParticleId[ev]) == 25
                    assert(np.count_nonzero(higgs_sel) == 1)
                    higgs = TLorentzVector()
                    higgs.SetPtEtaPhiE(gParticlePt[ev][higgs_sel],gParticleEta[ev][higgs_sel],gParticlePhi[ev][higgs_sel],gParticleE[ev][higgs_sel])
                    higgs_pt[k].append(gParticlePt[ev][higgs_sel][0])

            for gen_index in range(4): #gLLP daughters
                if gen_index == 0 or gen_index ==1:
                    llp_index = 0
                else:
                    llp_index = 1

                
                selection = gen_time_var[ev][gen_index] > -10 and photon_travel_time[ev][gen_index] > -10
#                 selection = selection and (not gLLP_daughter_match_jet_index[ev][gen_index] == 666)
                if selection:
                    gen_time[k].append(gen_time_var[ev][gen_index])
                    if not gLLP_daughter_match_jet_index[ev][gen_index] == 666:
                        gen_time_matched[k].append(gen_time_var[ev][gen_index])
                    decay_radius[k].append(gLLP_decay_radius[ev][llp_index])
                
                    delay_beta[k].append(gLLP_travel_time[ev][llp_index]*(1 - gLLP_beta[ev][llp_index]))
                    delay_path[k].append(gLLP_travel_time[ev][llp_index]*gLLP_beta[ev][llp_index] + gLLP_daughter_travel_time[ev][gen_index] - photon_travel_time[ev][gen_index])
#                     if not (gen_time_var[ev][gen_index] == genVertexT[ev]+gLLP_travel_time[ev][llp_index] + gLLP_daughter_travel_time[ev][gen_index] - photon_travel_time[ev][gen_index]):
#                         print(gen_time_var[ev][gen_index] , genVertexT[ev]+gLLP_travel_time[ev][llp_index] + gLLP_daughter_travel_time[ev][gen_index] - photon_travel_time[ev][gen_index])
                    
                    gLLP_daughter_deltaR[k].append(((gLLP_eta[ev][llp_index]-gLLP_daughter_eta[ev][gen_index])**2+(deltaPhi(gLLP_phi[ev][llp_index],gLLP_daughter_phi[ev][gen_index]))**2)**0.5)
                    llp_eta[k].append(gLLP_eta[ev][llp_index])
                    llp_phi[k].append(gLLP_phi[ev][llp_index])
                    llp_beta[k].append(gLLP_beta[ev][llp_index])
                    angle[k].append(LLP_daughter_angle[ev][gen_index])

                    gLLP_daughter = TLorentzVector()
                    gLLP = TLorentzVector()
                    #in lab frame
                    gLLP_daughter.SetPtEtaPhiE(gLLP_daughter_pt[ev][gen_index],gLLP_daughter_eta[ev][gen_index],gLLP_daughter_phi[ev][gen_index],gLLP_daughter_e[ev][gen_index])
                    gLLP.SetPtEtaPhiE(gLLP_pt[ev][llp_index],gLLP_eta[ev][llp_index],gLLP_phi[ev][llp_index],gLLP_e[ev][llp_index])
                    llp_p_lf[k].append(gLLP.Pt()*math.cosh(gLLP.Eta()))
                    #boost to llp rest frame
                    gLLP_daughter.Boost(-gLLP_beta_x[ev][llp_index],-gLLP_beta_y[ev][llp_index],-gLLP_beta_z[ev][llp_index])
                    gLLP.Boost(-gLLP_beta_x[ev][llp_index],-gLLP_beta_y[ev][llp_index],-gLLP_beta_z[ev][llp_index])
                    rf_angle[k].append(gLLP.Angle(gLLP_daughter.Vect()))
                    if k[:4] == 'bbbb':
                        #boost to higgs frame
                        gLLP.SetPtEtaPhiE(gLLP_pt[ev][llp_index],gLLP_eta[ev][llp_index],gLLP_phi[ev][llp_index],gLLP_e[ev][llp_index])
                        gLLP.Boost(-higgs.BoostVector())
                        llp_p_hf[k].append(gLLP.Pt()*math.cosh(gLLP.Eta()))
      

        gen_time[k] = np.array(gen_time[k])
        gen_time_matched[k] = np.array(gen_time_matched[k])


        delay_beta[k] = np.array(delay_beta[k])
        delay_path[k] = np.array(delay_path[k])
        gLLP_daughter_deltaR[k] = np.array(gLLP_daughter_deltaR[k])
        decay_radius[k] = np.array(decay_radius[k])
        llp_phi[k] = np.array(llp_phi[k])
        llp_eta[k] = np.array(llp_eta[k])
        llp_beta[k] = np.array(llp_beta[k])
        angle[k]=np.array(angle[k])
        rf_angle[k] = np.array(rf_angle[k])
        higgs_pt[k] = np.array(higgs_pt[k])
        llp_p_lf[k] = np.array(llp_p_lf[k])
        llp_p_hf[k] = np.array(llp_p_hf[k])






        print("mean of delay from beta", np.mean(delay_beta[k]))
        print("RMS of delay from beta",np.mean(delay_beta[k]**2)**0.5)
        print("mean of delay from path", np.mean(delay_path[k]))
        print("RMS of delay from path", np.mean(delay_path[k]**2)**0.5)
        print(np.count_nonzero(delay_beta[k]>5))
        print(k,len(gen_time[k]),len(gen_time_matched[k]))
        print("matching efficiency",len(gen_time_matched[k])*1.0/len(gen_time[k]))



mean of delay from beta 0.9053506003709271
RMS of delay from beta 1.4032952983417928
mean of delay from path 0.35954255
RMS of delay from path 0.8706439785829679
921
bbbb_ggh_mh125_ctau1 84441 29508
matching efficiency 0.34945109603154867
mean of delay from beta 0.7290232024698818
RMS of delay from beta 1.208812989129735
mean of delay from path 0.31967962
RMS of delay from path 0.7876693967249123
111
bbbb_mh125_ctau1 13982 5844
matching efficiency 0.41796595622943783
mean of delay from beta 1.9268047560001014
RMS of delay from beta 3.7270127704693268
mean of delay from path 0.4138317
RMS of delay from path 1.0770919937188976
279
bbbb_mh125_ctau10 3145 1174
matching efficiency 0.37329093799682034


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:66: RuntimeWarning: overflow encountered in sinh
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:67: RuntimeWarning: overflow encountered in cosh
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:69: RuntimeWarning: overflow encountered in cosh
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:70: RuntimeWarning: overflow encountered in cosh
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:71: RuntimeWarning: overflow encountered in sinh
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:71: RuntimeWarning: overflow encountered in cosh
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:71: RuntimeWarning: invalid value encountered in true_divide


mean of delay from beta 1.8118271448365149
RMS of delay from beta 2.596800518153541
mean of delay from path 0.35410115
RMS of delay from path 0.7625644781648071
434
bbbb_mh2000_ctau1 6880 5572
matching efficiency 0.8098837209302325
mean of delay from beta 5.779840992477863
RMS of delay from beta 9.088374882543075
mean of delay from path 0.6001454
RMS of delay from path 1.2545852011357677
977
bbbb_mh2000_ctau10 2479 1533
matching efficiency 0.6183945139169019
mean of delay from beta 1.3899583357219996
RMS of delay from beta 2.033314460780182
mean of delay from path 0.3034415
RMS of delay from path 0.7128958505935404
4861
gluino_m1000_ctau1 151148 62079
matching efficiency 0.41071664858284596
mean of delay from beta 3.9817203850421277
RMS of delay from beta 6.707417896708388
mean of delay from path 0.46180993
RMS of delay from path 1.118472340828
10046
gluino_m1000_ctau10 41897 15019
matching efficiency 0.3584743537723465
mean of delay from beta 0.8355210093406548
RMS of delay from beta 

In [21]:
# Plotting mass
c = rt.TCanvas('c','c', 800, 800)
h = {}
# leg = rt.TLegend(0.50,0.75,0.97,0.93)
leg = rt.TLegend(0.63,0.80,0.95,0.93)
leg.SetTextSize(0.022)
# leg.SetTextFont(42)
leg.SetEntrySeparation(0.01)

i = 0
max_y = 0
for k,v in tree_bkg.items():
#     print(v)
    if k== 'bbbb_mh125_mx55_ctau1':
        h['all'] = create_TH1D(gen_time[k], axis_title = ['gen time [ns]','Events'], binning=[25,0,25])
        h['matched'] = create_TH1D(gen_time_matched[k], axis_title = ['gen time [ns]','Events'], binning=[25,0,25])

        h['all'].SetLineColor(std_color_list[0])
        h['matched'].SetLineColor(std_color_list[1])

#         currentMax=h[k].GetBinContent(h[k].GetMaximumBin())

#         if currentMax > max_y:
#             max_y = currentMax
#             h[k].SetMaximum(10*currentMax)
#             h[k].SetMinimum(1)
#         h[k].GetXaxis().SetLabelSize(0.040)
        
#         h[k].GetXaxis().SetTickLength(0.)
        leg.AddEntry(h['all'], 'All LLP daughters')
        leg.AddEntry(h['matched'], 'Matched LLP daughters')

        h['all'].Draw('')
        h['matched'].Draw('same')

# c.SetRightMargin(0.17)

leg.Draw()


c.SetLogy()
c.Draw()
c.SaveAs('../plots/test.pdf')




Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Info in <TCanvas::Print>: pdf file ../plots/test.pdf has been created


In [52]:
# Plotting mass
c = rt.TCanvas('c','c', 800, 800)
h = {}
# leg = rt.TLegend(0.50,0.75,0.97,0.93)
leg = rt.TLegend(0.65,0.80,0.95,0.93)
leg.SetTextSize(0.022)
# leg.SetTextFont(42)
leg.SetEntrySeparation(0.01)

i = 0
max_y = 0
for k,v in tree_bkg.items():
#     print(v)
    if k[:4]== 'bbbb':
        print(higgs_pt[k].shape)
#         h[k] = create_TH1D(higgs_pt[k], axis_title = ['Higgs pT','Events'], binning=[50,0,500])
#         h[k] = create_TH1D(llp_p_hf[k]/llp_mass[k], axis_title = ['p_{LLP}/m_{LLP}','Events'], binning=[30,0,3])
        h[k] = create_TH1D(gen_time[k], axis_title = ['Gen Time [ns]','Events'], binning=[25,-1,50])







        currentMax=h[k].GetBinContent(h[k].GetMaximumBin())

        if currentMax > max_y:
            max_y = currentMax
            h[k].SetMaximum(10*currentMax)
            h[k].SetMinimum(1)
        h[k].GetXaxis().SetLabelSize(0.040)
        
#         h[k].GetXaxis().SetTickLength(0.)
        h[k].SetLineColor(std_color_list[i])
        leg.AddEntry(h[k], k)
        h[k].DrawNormalized('same')
        i+=1

# c.SetRightMargin(0.17)

leg.Draw()


c.SetLogy()
c.Draw()
c.SaveAs('../plots/test.pdf')



(50918,)
(10545,)
(11654,)
(4193,)
(4207,)
(16747,)


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Info in <TCanvas::Print>: pdf file ../plots/test.pdf has been created


In [53]:
# Plotting mass
c = rt.TCanvas('c','c', 800, 800)
h = {}
# leg = rt.TLegend(0.50,0.75,0.97,0.93)
leg = rt.TLegend(0.65,0.80,0.95,0.93)
leg.SetTextSize(0.022)
# leg.SetTextFont(42)
leg.SetEntrySeparation(0.01)

i = 0
max_y = 0
for k,v in tree_bkg.items():
#     print(v)
    if k[:4] == 'bbbb' or k[:4] == 'glui':


#         h[k] = create_TH1D(gLLP_daughter_deltaR[k], axis_title = ['#Delta R','Events'], binning=[25,0,8])
#         h[k] = create_TH1D(np.cos(rf_angle[k]), axis_title = ['cos (#theta)','Events'], binning=[25,-1,1])
#         h[k] = create_TH1D(rf_angle[k]*180/math.pi, axis_title = ['#theta [#circ]','Events'], binning=[30,0,180])
#         h[k] = create_TH1D(angle[k], axis_title = ['cos (#theta)','Events'], binning=[30,-1,1]) #lab frame cosine
#         h[k] = create_TH1D(np.arccos(angle[k])*180/math.pi, axis_title = ['#theta [#circ]','Events'], binning=[30,0,180])

#         h[k] = create_TH1D(llp_eta[k], axis_title = ['LLP #eta','Events'], binning=[25,-5,5])
#         h[k] = create_TH1D(llp_phi[k], axis_title = ['LLP #phi','Events'], binning=[30,-4,4])
        h[k] = create_TH1D(llp_beta[k], axis_title = ['LLP #beta','Events'], binning=[30,0,1])




#         h[k] = create_TH1D(llp_p_lf[k]/llp_mass[k], axis_title = ['p_{LLP}/m_{LLP}','Events'], binning=[30,0,10])
#         h[k] = create_TH1D(gen_time[k], axis_title = ['Gen Time [ns]','Events'], binning=[25,-1,50])


        currentMax=h[k].GetBinContent(h[k].GetMaximumBin())

        if currentMax > max_y:
            max_y = currentMax
            h[k].SetMaximum(10*currentMax)
            h[k].SetMinimum(1)
        h[k].GetXaxis().SetLabelSize(0.045)
        
#         h[k].GetXaxis().SetTickLength(0.)
        h[k].SetLineColor(std_color_list[i])
        leg.AddEntry(h[k], k)
        h[k].DrawNormalized('same')
        i+=1

# c.SetRightMargin(0.17)

leg.Draw()


c.SetLogy()
c.Draw()
c.SaveAs('../plots/test.pdf')





Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Info in <TCanvas::Print>: pdf file ../plots/test.pdf has been created


In [47]:
# Plotting mass
c = rt.TCanvas('c','c', 900, 600)
h = {}
# leg = rt.TLegend(0.50,0.75,0.97,0.93)
leg = rt.TLegend(0.58,0.70,0.97,0.93)
leg.SetTextSize(0.022)
# leg.SetTextFont(42)
leg.SetEntrySeparation(0.01)

i = 0
for k,v in tree_bkg.items():
#     print(v)
#     if k == 'bbbb_mh2000_ctau1':
    if k[:4] == 'bbbb' or k[:4] == 'glui':
#         print(np.mean(delay_beta[k]))
#         print(np.mean(delay_beta[k]**2)**0.5)
#         print(np.mean(delay_path[k]))
#         print(np.mean(delay_path[k]**2)**0.5)

        h[k] = create_TH2D(np.column_stack((delay_path[k],delay_beta[k])), axis_title = ['Delay from Path [ns]','Delay from #beta [ns]','Events'], binning=[50,0,20,50,0,20])

        h[k].SetLineColor(std_color_list[i])
        leg.AddEntry(h[k], k)
        h[k].Draw('colz')
        i+=1

    c.SetRightMargin(0.17)

    # leg.Draw()


    c.SetLogz()
    c.Draw()
    c.SaveAs('../plots/delay_contribution_'+k+'.pdf')
c.SaveAs('../plots/test.pdf')


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: pdf file ../plots/delay_contribution_QCD.pdf has been created
Info in <TCanvas::Print>: pdf file ../plots/delay_contribution_WJetsToLNu.pdf has been created
Info in <TCanvas::Print>: pdf file ../plots/delay_contribution_bbbb_mh125_ctau1.pdf has been created
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Info in <TCanvas::Print>: pdf file ../plots/delay_contribution_bbbb_mh125_ctau10.pdf has been created
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Info in <TCanvas::Print>: pdf file ../plots/delay_contribution_bbbb_mh2000_ctau1.pdf has been created
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Info in <TCanvas::Print>: pdf file ../plots/delay_contribution_bbbb_mh2000_ctau10.pdf has been created
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Info in <TCanvas::Print>

In [36]:
# Plotting mass

for k,v in tree_bkg.items():
#     print(v)
    if k == 'bbbb_mh2000_ctau1':
#     if k == 'WJetsToLNu':
#     if k == ''
        for index in range(len(rechits_time[k])):
#             if index == 10:
            if reco_time[k][index]>4 and jet_n_rechits_Ecut0p5[k][index]>5:
                print(reco_time[k][index])
    #         print(len(rechits_eta[k]))
    #         index =500
    #         print(len(rechits_eta[k][index]))
    #         print(rechits_phi[k][index])
                c = rt.TCanvas('c','c', 800, 600)
                h = {}

                bins = [170, -1.48, 1.48, 360, -3.14, 3.14]
        #         bins = [80,-4,4,80,-4,4]
        #         print(np.transpose(np.row_stack((np.array(rechits_eta[k][index]),np.array(rechits_phi[k][index])))).shape)
                print(np.transpose(rechits_time[k][index][0]).shape)
                print(rechits_time[k][index])
                h[k] = create_TH1D(np.transpose(rechits_time[k][index][0]), axis_title = ['ECAL rechit time [ns]','Events'], binning=[500,0,5])

                h[k].SetMaximum(2.0)
                h[k].SetMinimum(0.0)
                h[k].Draw('colz')

                c.SetRightMargin(0.17)

            # leg.Draw()


        #     c.SetLogz()
                c.Draw()
                break
#             c.SaveAs('../plots/rechit_map/2d_'+k+'_'+str(index)+'.pdf')



NameError: name 'rechits_time' is not defined

In [ ]:
# Plotting mass

for k,v in tree_bkg.items():
#     print(v)
#     if k == 'bbbb_mh125_ctau1':
#     if k == 'WJetsToLNu':
    for index in range(len(rechits_eta[k])):
#         print(len(rechits_eta[k]))
#         index =500
#         print(len(rechits_eta[k][index]))
#         print(rechits_phi[k][index])
        c = rt.TCanvas('c','c', 800, 800)
        h = {}
        
        bins = [170, -1.48, 1.48, 360, -3.14, 3.14]
#         bins = [80,-4,4,80,-4,4]
#         print(np.transpose(np.row_stack((np.array(rechits_eta[k][index]),np.array(rechits_phi[k][index])))).shape)
        h[k] = create_TH2D(np.transpose(np.row_stack((rechits_eta[k][index],rechits_phi[k][index]))), axis_title = ['#eta','#phi','Events'], binning=bins)
        h[k].SetMaximum(2.0)
        h[k].SetMinimum(1.0)
        h[k].Draw('colz')

        c.SetRightMargin(0.17)

    # leg.Draw()


#     c.SetLogz()
        c.Draw()
        c.SaveAs('../plots/rechit_map/2d_'+k+'_'+str(index)+'.pdf')


In [ ]:
# for k in tree_bkg.keys():
#     print (k)
#     rechits_cuts = [0,5,10,15,20,25]
#     time_cut = 1
#     for cut in rechits_cuts:
#         rejection = 1.0* len(reco_time[k][reco_time[k]>1])/len(jet_n_rechits[k][np.logical_and(jet_n_rechits[k]>cut,reco_time[k]>1)])
#         print("rejection power for time > 1ns, with rechit cut "+str(cut)+":"+str(rejection))
#     print("\n")
#     rechits_cuts = [0,1,2,3,4,5,6,7,8,9,10]
#     time_cut = 1
#     for cut in rechits_cuts:
#         rejection = 1.0* len(reco_time[k][reco_time[k]>1])/len(jet_n_rechits_Ecut1[k][np.logical_and(jet_n_rechits_Ecut1[k]>cut,reco_time[k]>1)])
#         print("rejection power for time > 1ns, with rechit_Ecut1 cut "+str(cut)+":"+str(rejection))
#     print("\n")
#     rechits_cuts = [0,0.5,0.6,0.7,0.8,1]
#     time_cut = 1
#     for cut in rechits_cuts:
#         rejection = 1.0* len(reco_time[k][reco_time[k]>1])/len(e_over_h[k][np.logical_and(e_over_h[k]>cut,reco_time[k]>1)])
#         print("rejection power for time > 1ns, with E/H cut "+str(cut)+":"+str(rejection))
#     print("\n")
    
#     rechits_cuts = [1,1.5,2,2.5,3]

#     for cut in rechits_cuts:
#         rejection = 1.0* len(reco_time[k][reco_time[k]>1])/len(rms[k][np.logical_and(rms[k]<cut,reco_time[k]>1)])
#         print("rejection power for time > 1ns, with jetTime RMS cut "+str(cut)+":"+str(rejection))
#     print("\n")


In [ ]:
# rechit vs time


i = 0
h = {}
c = rt.TCanvas('c','c', 800, 800)

leg = rt.TLegend(0.55,0.81,0.95,0.93)
# leg = rt.TLegend(0.50,0.82,0.80,0.93)
leg.SetTextSize(0.022)
# leg.SetTextFont(42)
leg.SetEntrySeparation(0.01)


k = 'WJetsToLNu'
h['0p5'] = create_TH1D(reco_time[k], axis_title = ['Gen Time [ns]','Events'], binning=[50,-10,20])
h['1'] = create_TH1D(reco_time_Ecut1[k], axis_title = ['Gen Time [ns]','Events'], binning=[50,-10,20])

# h['gen'] = create_TH2D(np.column_stack((gen_time[k]-reco_time[k],reco_time[k])), axis_title = ['Difference','Gen Time [ns]','Events'], binning=[50,-15,15,50,-15,50])
h['1'].SetLineColor(std_color_list[1])
h['0p5'].SetLineColor(std_color_list[0])

# currentMax=h[k].GetBinContent(h[k].GetMaximumBin())

# if currentMax > max_y:
#     max_y = currentMax
#     h[k].SetMaximum(3*currentMax)
#     h[k].SetMinimum(1)
leg.AddEntry(h['1'], '1')
leg.AddEntry(h['0p5'], '0p5')

h['0p5'].Draw('')
h['1'].Draw('same')


# c.SetRightMargin(0.17)


leg.Draw()


c.SetLogy()
c.Draw()

#     c.SaveAs('../plots/'+name[var_i]+'.pdf')

In [ ]:
# rechit vs time


i = 0
h = {}
c = rt.TCanvas('c','c', 800, 800)

leg = rt.TLegend(0.58,0.81,0.95,0.93)
# leg = rt.TLegend(0.50,0.82,0.80,0.93)
leg.SetTextSize(0.022)
# leg.SetTextFont(42)
leg.SetEntrySeparation(0.01)
keys = ['bbbb_mh125_ctau1','bbbb_mh125_ctau10','bbbb_mh2000_ctau1']
i = 0
k = 'bbbb_mh2000_ctau1'
# print(gen_time[k])
# print(reco_time[k])
bins = [40,-5,25]
# bins = [40,-8,20]
h['gen'] = create_TH1D(gen_time[k], axis_title = ['Time [ns]','Events'], binning=bins)
h['reco'] = create_TH1D(reco_time[k], axis_title = ['Time [ns]','Events'], binning=bins)

#     h['gen'].SetLineColor(std_color_list[0])
#     h['reco'].SetLineColor(std_color_list[1])
h['gen'].SetLineColor(std_color_list[0])
h['reco'].SetLineColor(std_color_list[1])

i+=1
leg.AddEntry(h['gen'], 'Gen Time')
leg.AddEntry(h['reco'],'Reco Time')
# leg.AddEntry(h['response'],'Time Response')

# h['gen'].DrawNormalized('same')
# h['reco'].DrawNormalized('same')
print(k)
#     f = h[k].Fit("gaus")
h['gen'].DrawNormalized('same')
h['reco'].DrawNormalized('same')
#     f.Draw()

# c.SetRightMargin(0.17)


leg.Draw()


c.SetLogy()
c.Draw()




#     c.SaveAs('../plots/'+name[var_i]+'.pdf')

In [ ]:
# rechit vs time


i = 0
h = {}
c = rt.TCanvas('c','c', 800, 800)

leg = rt.TLegend(0.58,0.81,0.95,0.93)
# leg = rt.TLegend(0.50,0.82,0.80,0.93)
leg.SetTextSize(0.022)
# leg.SetTextFont(42)
leg.SetEntrySeparation(0.01)
keys = ['bbbb_mh125_ctau1','bbbb_mh125_ctau10','bbbb_mh2000_ctau1']
i = 0
rebin = 100000
for k in keys:
# k = 'bbbb_mh2000_ctau1'
# print(gen_time[k])
# print(reco_time[k])

    bins = [40*rebin,-8,10]
#     h['gen'] = create_TH1D(gen_time[k], axis_title = ['Time [ns]','Events'], binning=bins)
#     h['reco'] = create_TH1D(reco_time[k], axis_title = ['Time [ns]','Events'], binning=bins)
    h[k] = create_TH1D(reco_time[k]-gen_time[k], axis_title = ['t_{reco}-t_{gen} [ns]','Events'], binning=bins)

#     h['gen'].SetLineColor(std_color_list[0])
#     h['reco'].SetLineColor(std_color_list[1])
    h[k].SetLineColor(std_color_list[i])
    i+=1
    leg.AddEntry(h[k],k)
#     leg.AddEntry(h['gen'], 'Gen Time')
#     leg.AddEntry(h['reco'],'Reco Time')
    # leg.AddEntry(h['response'],'Time Response')

    # h['gen'].DrawNormalized('same')
    # h['reco'].DrawNormalized('same')
    print(k)
#     f = h[k].Fit("gaus")
    print(sigma_effective(h[k]))
    h[k].Rebin(rebin)
    h[k].DrawNormalized('same')

#     f.Draw()

# c.SetRightMargin(0.17)


leg.Draw()


c.SetLogy()
c.Draw()




#     c.SaveAs('../plots/'+name[var_i]+'.pdf')

In [ ]:
def sigma_effective(h1):

    r = h1.Fit('gaus', 'LRSQ+', '', -2, 2)
    r = r.Get()
   
    n_pk = h1.FindBin(r.Parameter(1))
    thr = 0.68 * h1.Integral()
    counter = h1.GetBinContent(n_pk)*1.0
    n_low = n_pk
    n_up = n_pk
    flag = True
    while counter < thr:
        if flag:
            n_low -= 1
            counter += h1.GetBinContent(n_low)
        else:
            n_up += 1
            counter += h1.GetBinContent(n_up)
        flag = not flag
    return (n_up - n_low) *h1.GetBinWidth(1)


In [ ]:
# rechit vs time


i = 0
h = {}
c = rt.TCanvas('c','c', 800, 800)

leg = rt.TLegend(0.55,0.81,0.95,0.93)
# leg = rt.TLegend(0.50,0.82,0.80,0.93)
leg.SetTextSize(0.022)
# leg.SetTextFont(42)
leg.SetEntrySeparation(0.01)

for k,v in gen_time.items():
#     print(v)
    if k[:4] == 'bbbb':
        h[k] = create_TH1D(v, axis_title = ['Gen Time [ns]','Events'], binning=[30,-5,15])
        h[k].SetLineColor(std_color_list[i])
        currentMax=h[k].GetBinContent(h[k].GetMaximumBin())

        if currentMax > max_y:
            max_y = currentMax
            h[k].SetMaximum(3*currentMax)
            h[k].SetMinimum(1)

        leg.AddEntry(h[k], k)
        h[k].DrawNormalized('same')
        i+=1

        # c.SetRightMargin(0.17)


        leg.Draw()


        c.SetLogy()
        c.Draw()

#     c.SaveAs('../plots/'+name[var_i]+'.pdf')

In [ ]:
# rechit vs time
var = [reco_time,chef,ceef,rms_frac,ecalE,jet_n_rechits_Ecut0p5,h_over_e,h_fraction,em_fraction,rms,charged_frac,charged_m,neutral_m]
xaxis = ['Reco Time [ns]','Charged Hadron Energy Fraction','Charged EM Energy Fraction','t^{RMS}_{jet}/t_{jet}','ECAL Energy [Gev]','N_{ECAL}^{cell}','H/E','HEF','EEF','jetTime RMS [ns]','Charged Energy Fraction','Charged Hadron Multiplicity','Neutral Hadron Multiplicity']
bins = [[30,-5,10],[20,0,1],[20,0,1],[25,0,10],[25,0,100],[25,0,200],[20,0,1],[20,0,1],[20,0,1],[25,0,10],[20,0,1],[20,0,40],[20,0,40]]
name = ['reco_time','chef','ceef','rms_recotime_ratio','ecalE','jet_n_rechits_Ecut0p5','h_over_e','h_fraction','em_fraction','rms','charged_frac','charged_m','neutral_m']
bins = [[30,-10,20],[20,0,1],[20,0,1],[25,0,5],[25,0,500],[50,0,200],[20,0,1],[10,0,1],[20,0,1],[25,0,10],[20,0,1],[20,0,40],[20,0,40]]
assert(len(var)==len(xaxis)==len(bins)==len(name))

# var_i = 1
for var_i in range(len(var)):
    if var_i ==6:
        i = 0
        h = {}
        c = rt.TCanvas('c','c', 800, 800)

        leg = rt.TLegend(0.55,0.81,0.95,0.93)
        # leg = rt.TLegend(0.50,0.82,0.80,0.93)
        leg.SetTextSize(0.022)
        # leg.SetTextFont(42)
        leg.SetEntrySeparation(0.01)
        max_y = 0
        for k,v in tree_bkg.items():
        #     print(v)
            if not k == "QCD" or True:
#             if k[:4] == 'bbbb' or True:
                h[k] = create_TH1D(var[var_i][k], axis_title = [xaxis[var_i],'Events'], binning=bins[var_i])
                h[k].SetLineColor(std_color_list[i])
                currentMax=h[k].GetBinContent(h[k].GetMaximumBin())

                if currentMax > max_y:
                    max_y = currentMax
                    h[k].SetMaximum(50*currentMax)
                    h[k].SetMinimum(1)

                leg.AddEntry(h[k], k)
                h[k].DrawNormalized('same')
                i+=1

        # c.SetRightMargin(0.17)


        leg.Draw()


        c.SetLogy()
        c.Draw()

#     c.SaveAs('../plots/'+name[var_i]+'.pdf')

In [ ]:
# rechit vs time
var = [reco_time,chef,ceef,rms_frac,ecalE,jet_n_rechits_Ecut0p5,h_over_e,h_fraction,em_fraction,rms,charged_frac,charged_m,neutral_m]
xaxis = ['Reco Time [ns]','Charged Hadron Energy Fraction','Charged EM Energy Fraction','t^{RMS}_{jet}/t_{jet}','ECAL Energy [Gev]','N_{ECAL}^{cell}','H/E','HEF','EEF','jetTime RMS [ns]','Charged Energy Fraction','Charged Hadron Multiplicity','Neutral Hadron Multiplicity']
bins = [[30,-5,10],[20,0,1],[20,0,1],[25,0,10],[25,0,100],[25,0,200],[20,0,1],[20,0,1],[20,0,1],[25,0,10],[20,0,1],[20,0,40],[20,0,40]]
name = ['reco_time','chef','ceef','rms_recotime_ratio','ecalE','jet_n_rechits_Ecut0p5','h_over_e','h_fraction','em_fraction','rms','charged_frac','charged_m','neutral_m']
bins = [[30,-10,20],[20,0,1],[20,0,1],[25,0,5],[25,0,500],[50,0,200],[20,0,1],[20,0,1],[20,0,1],[25,0,10],[20,0,1],[20,0,40],[20,0,40]]
assert(len(var)==len(xaxis)==len(bins)==len(name))

for var_i in range(len(var)):
    
    if var_i == 3 or True:
        c = rt.TCanvas('c','c', 900, 800)
        h = {}
        leg = rt.TLegend(0.58,0.70,0.97,0.93)
        leg.SetTextSize(0.022)
        leg.SetEntrySeparation(0.01)
        i = 0
        for k,v in tree_bkg.items():
        #     print(v)
        #     if k == 'WJetsToLNu':
            if k == 'bbbb_mh2000_ctau1':
                h[k] = create_TH2D(np.column_stack((reco_time[k],var[var_i][k])), axis_title = ['Reco time [ns]',xaxis[var_i],'Events'], binning=[30,-15,15]+bins[var_i])
                h[k].SetLineColor(std_color_list[i])
                h[k].DrawNormalized('colz')

                c.SetRightMargin(0.17)

        # leg.Draw()


                c.SetLogz()
                c.Draw()
#                 c.SaveAs('../plots/2d_'+k+'_'+name[var_i]+'.pdf')


In [ ]:
# Plotting mass
c = rt.TCanvas('c','c', 800, 800)
h = {}
# leg = rt.TLegend(0.50,0.75,0.97,0.93)
leg = rt.TLegend(0.58,0.70,0.97,0.93)
leg.SetTextSize(0.022)
# leg.SetTextFont(42)
leg.SetEntrySeparation(0.01)

i = 0
for k,v in tree_bkg.items():
#     print(v)
    if k == 'bbbb_mh2000_ctau1':
        print(h_over_e[k].shape)
        print(decay_radius[k].shape)
        h[k] = create_TH2D(np.column_stack((decay_radius[k],h_over_e[k])), axis_title = ['Decay Radius [cm]','H/E','Events'], binning=[25,0,130,50,0,10])

#         h[k] = create_TH2D(np.column_stack((decay_radius[k],charged_frac[k])), axis_title = ['Decay Radius [cm]','Charged Energy Fraction','Events'], binning=[25,0,130,25,0,1])
        h[k].SetLineColor(std_color_list[i])
        leg.AddEntry(h[k], k)
        h[k].Draw('colz')
        i+=1

    c.SetRightMargin(0.17)

    # leg.Draw()


    c.SetLogz()
    c.Draw()
    c.SaveAs('../plots/2d_'+k+'_'+'chargedfrac_hovere.pdf')


In [ ]:
# Plotting mass
c = rt.TCanvas('c','c', 800, 800)
h = {}
# leg = rt.TLegend(0.50,0.75,0.97,0.93)
leg = rt.TLegend(0.58,0.70,0.97,0.93)
leg.SetTextSize(0.022)
# leg.SetTextFont(42)
leg.SetEntrySeparation(0.01)

i = 0
for k,v in tree_bkg.items():
#     print(v)
#     if k == 'QCD':
    if k == 'bbbb_mh2000_ctau10':
        h[k] = create_TH2D(np.column_stack((charged_frac[k],h_over_e[k])), axis_title = ['CEF','H/E','Events'], binning=[50,0,1,50,0,15])
        h[k].SetLineColor(std_color_list[i])
        leg.AddEntry(h[k], k)
        h[k].Draw('colz')
        i+=1

c.SetRightMargin(0.17)

# leg.Draw()


c.SetLogz()
c.Draw()



In [ ]:
# Plotting mass
c = rt.TCanvas('c','c', 800, 800)
h = {}
# leg = rt.TLegend(0.50,0.75,0.97,0.93)
leg = rt.TLegend(0.58,0.70,0.97,0.93)
leg.SetTextSize(0.022)
# leg.SetTextFont(42)
leg.SetEntrySeparation(0.01)

i = 0
for k,v in tree_bkg.items():
#     print(v)
    if k == 'bbbb':
        h[k] = create_TH2D(np.column_stack((reco_time[k],h_over_e[k])), axis_title = ['Reco time [ns]','H/E','Events'], binning=[50,-20,20,50,0,5])
        h[k].SetLineColor(std_color_list[i])
        leg.AddEntry(h[k], k)
        h[k].Draw('colz')
        i+=1

c.SetRightMargin(0.17)

# leg.Draw()

# c.SetLogy()

c.SetLogz()
c.Draw()



In [ ]:
# rechit vs time
c = rt.TCanvas('c','c', 800, 800)
h = {}
# leg = rt.TLegend(0.50,0.75,0.97,0.93)
leg = rt.TLegend(0.58,0.70,0.97,0.93)
leg.SetTextSize(0.022)
# leg.SetTextFont(42)
leg.SetEntrySeparation(0.01)

i = 0
for k,v in tree_bkg.items():
#     print(v)
    if not k == 'bbbb':
        h[k] = create_TH2D(np.column_stack((reco_time[k],rms[k])), axis_title = ['Reco Time [ns]','jetTime RMS [ns]','Events'], binning=[50,-20,20,50,0,2])
        h[k].SetLineColor(std_color_list[i])
        leg.AddEntry(h[k], k)
        h[k].Draw('colz')
        i+=1

c.SetRightMargin(0.17)

# leg.Draw()


c.SetLogz()
c.Draw()



In [ ]:
# rechit vs time
c = rt.TCanvas('c','c', 800, 800)
h = {}
# leg = rt.TLegend(0.50,0.75,0.97,0.93)
leg = rt.TLegend(0.58,0.70,0.97,0.93)
leg.SetTextSize(0.022)
# leg.SetTextFont(42)
leg.SetEntrySeparation(0.01)

i = 0
for k,v in tree_bkg.items():
#     print(v)
    h[k] = create_TH2D(np.column_stack((reco_time,std)), axis_title = ['Reco time [ns]','std','Events'], binning=[50,-20,20,100,0,50])
    h[k].SetLineColor(std_color_list[i])
    leg.AddEntry(h[k], k)
    h[k].Draw('colz')
    i+=1

c.SetRightMargin(0.17)

# leg.Draw()


c.SetLogz()
c.Draw()

